# FITS File Editor

The main focus of this tutorial is to demonstrate how to create a new Astropy Table and save the data in a new FITS file.

In [1]:
import numpy as np
from astropy.table import Table, join
import pandas as pd

Read in the files as Astropy Tables.

In [2]:
pd.set_option('display.max_columns', None)

filename_1 = r"filepath/filename.fits"
filename_2 = "filepath/filename.fits"

catalog_matcha = Table.read(filename_1)
catalog_xmm = Table.read(filename_2)

In [3]:
print(f"The number of columns in `catalog_matcha` are {len(catalog_matcha.colnames)}")
print(f"The number of rows are {len(catalog_matcha)}.")
catalog_matcha[0:5]

The number of columns in `catalog_matcha` are 110
The number of rows are 1092.


Name,MEM_MATCH_ID,Analyzed,Detected,redMaPPer_ra,redMaPPer_dec,Redshift,Obsids,lambda,lambda_err_low,lambda_err_high,band_lumin_upper_lim_undetected,band_lumin_upper_lim_detected,lumin_no_tx,lumin_no_tx_err_low,lumin_no_tx_err_high,x_ray_peak_ra,x_ray_peak_dec,500_kiloparsecs_ra,500_kiloparsecs_dec,500_kiloparsecs_temperature,500_kiloparsecs_temperature_err_low,500_kiloparsecs_temperature_err_high,500_kiloparsecs_band_lumin,500_kiloparsecs_band_lumin_err_low,500_kiloparsecs_band_lumin_err_high,500_kiloparsecs_bolo_lumin,500_kiloparsecs_bolo_lumin_err_low,500_kiloparsecs_bolo_lumin_err_high,500_kiloparsecs_radius,500_kiloparsecs_radius_err_low,500_kiloparsecs_radius_err_high,500_kiloparsecs_SNR,500_kiloparsecs_SNR_err_low,500_kiloparsecs_SNR_err_high,r2500_ra,r2500_dec,r2500_temperature,r2500_temperature_err_low,r2500_temperature_err_high,r2500_band_lumin,r2500_band_lumin_err_low,r2500_band_lumin_err_high,r2500_bolo_lumin,r2500_bolo_lumin_err_low,r2500_bolo_lumin_err_high,r2500_radius,r2500_radius_err_low,r2500_radius_err_high,r2500_SNR,r2500_SNR_err_low,r2500_SNR_err_high,r500_ra,r500_dec,r500_temperature,r500_temperature_err_low,r500_temperature_err_high,r500_band_lumin,r500_band_lumin_err_low,r500_band_lumin_err_high,r500_bolo_lumin,r500_bolo_lumin_err_low,r500_bolo_lumin_err_high,r500_radius,r500_radius_err_low,r500_radius_err_high,r500_SNR,r500_SNR_err_low,r500_SNR_err_high,r500_core_cropped_ra,r500_core_cropped_dec,r500_core_cropped_temperature,r500_core_cropped_temperature_err_low,r500_core_cropped_temperature_err_high,r500_core_cropped_band_lumin,r500_core_cropped_band_lumin_err_low,r500_core_cropped_band_lumin_err_high,r500_core_cropped_bolo_lumin,r500_core_cropped_bolo_lumin_err_low,r500_core_cropped_bolo_lumin_err_high,r500_core_cropped_radius,r500_core_cropped_radius_err_low,r500_core_cropped_radius_err_high,r500_core_cropped_SNR,r500_core_cropped_SNR_err_low,r500_core_cropped_SNR_err_high,peak_fix,masked,bad_mode,quick look bad,obsid_bad,merger,Serendipitous,edge_centroid,edge_peak,edge_r2500,edge_r500,edge_bkgd,overlap_r2500,overlap_r500,overlap_bkgd,notes,Group 1 Check,redmapper wrong,right RA,right DEC,right in top 5,right not member,ambiguous,Chandra Notes
bytes15,int64,bool,bool,float64,float64,float64,bytes96,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes4,int64,int64,bytes4,bytes26,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,bytes106,bytes14,int64,float64,float64,int64,int64,int64,bytes145
catalogue_1,2,True,True,83.231901970306,-37.0266894362176,0.2833377420902252,15112,199.432,5.87942,5.87942,nan,nan,4.5843e+44,4.815e+42,4.8632e+42,83.2325,-37.02649,83.23211,-37.02644,11.344,0.68483,0.68878,4.3399e+44,6.51e+42,6.5277e+42,1.7469e+45,2.866e+43,2.9201e+43,500.0,0.0,0.0,94.277,1.087,1.087,83.23188,-37.02643,11.166,0.63832,0.6424,4.944e+44,6.9038e+42,6.8134e+42,1.9782e+45,3.086e+43,3.0796e+43,672.57,23.048,22.571,101.61,1.093,1.093,83.23226,-37.02672,10.934,0.85751,0.85747,5.3675e+44,8.55e+42,8.5518e+42,2.1305e+45,4.5892e+43,4.6121e+43,1511.0,49.857,48.953,79.355,1.039,1.039,83.23226,-37.02672,12.746,1.8843,2.0198,2.7238e+44,6.0129e+42,8.2416e+42,1.1614e+45,6.8477e+43,2.9554e+43,1511.0,49.857,48.953,46.384,1.0189,1.0189,N/A,0,0,N/A,N/A,0,0,0,0,0,0,0,0,0,0,N/A,yes,0,9999.0,9999.0,0,0,0,N/A
catalogue_2,3,True,True,347.092550205901,-2.19213688831977,0.29250583052635193,"4962, 9372",163.583,4.29459,4.29459,nan,3.406091810231530

In [4]:
print(f"The number of columns in `catalog_xmm` are {len(catalog_xmm.colnames)}")
print(f"The number of rows are {len(catalog_xmm)}.")
catalog_xmm[0:5]

The number of columns in `catalog_xmm` are 47
The number of rows are 200.


MEM_MATCH_ID,Target,XCS_NAMES,XCS_NAMES_2,col1,Unnamed: 0.1,RA,DEC,Z_1,LAMBDA_CHISQ,LAMBDA_CHISQ_E,R_LAMBDA,DEC_xmm,DISTANCE,EXP,ID,MEDIAN_EXP,OBSID,RA_xmm,WORST_EXP,WORST_MEDIAN_EXP,RA_ext,DEC_ext,P_DIS,COUNTS,SURVEY,R_LAM,LAM,Pk2RM,Pk2RM_kpc,Pk2RM2,Pk2RM2_kpc,RA_pk,DEC_pk,#XCS_NAME,z_2,Tx,Tx_m,Tx_p,Lx,Lx_m,Lx_p,Lx52,Lx52_m,Lx52_p,pnsn,pnsnr_alt
int32,int32,str25,str24,int32,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,float64,int32,float64,float64,float64,float64,float64,float64,float64,str6,float64,float64,float64,float64,float64,float64,float64,float64,str24,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1,1,XMMXCS J025417.8-585705.2,XMMXCSJ025417.8-585705.2,0,0,43.56457918170071,-58.95295900289761,0.4281561,221.67424,5.712534,1.1725816,-58.9771388888889,0.04642919334271176,74854.9296875,1,74694.171875,674380301,43.641458333333304,61328.166015625,64847.056640625,43.574507000000004,-58.951461,107.47517501267971,32295.22700000001,DES_Y3,1.1725816,221.67424,0.2473545738116952,83.05160455901164,0.2473545738116952,83.0516045590119,43.570019454392956,-58.949938643525904,XMMXCSJ025417.8-585705.2,0.428,7.91046,7.730295932,8.091945509,13.01380269,12.840220227904199,13.202123201528039,2.997567288,2.9770913590619004,3.018127763498787,13.1179,9.86521
2,1,XMMXCS J053256.0-370133.3,XMMXCSJ053256.0-370133.3,1,1,83.231901970306,-37.0266894362176,0.2871882,199.4325,5.87942,1.1480458,-36.998722222222206,0.028564471102779813,16632.501953125,2,16635.34033203125,42341801,83.224625,13579.380859375,14749.26318359375,83.233448,-37.025921999999994,22.60739375053894,16973.586,DES_Y3,1.1480458,199.4325,0.15772399477972346,40.88901232471884,0.15772399477972346,40.889012324719054,83.23447183176867,-37.0283329120189,XMMXCSJ053256.0-370133.3,0.287,7.828,7.609773848,8.048303197,19.62732649,19.305081542555765,19.93873807950439,4.548680698,4.510596719107621,4.584063988078493,53.7371,50.3601
3,1,XMMXCS J230821.8-021127.4,XMMXCSJ230821.8-021127.4,2,2,347.09255020590103,-2.1921368883197703,0.29409084,163.5833,4.2945933,1.1034377,-2.17591666666667,0.02564910414517177,15113.7470703125,3,15856.80810546875,205330501,347.0726666666671,13838.880859375,14390.849365234375,347.09112999999996,-2.1909587999999998,29.17054723486477,9638.7589,DES_Y3,1.1034377,163.5833,0.007471917725296069,1.9695313121040758,0.007471917725296069,1.9695313121040807,347.09262570545286,-2.192235965903757,XMMXCSJ230821.8-021127.4,0.294,8.1871,7.876648882,8.50001011,12.34927803,12.05060771693234,12.64424994913659,2.788004251,2.757385067135689,2.819077010660077,42.4223,39.7919
4,1,XMMXCS J051636.6-543120.8,XMMXCSJ051636.6-543120.8,3,3,79.15568493065871,-54.5004588211563,0.29913905,207.24304,7.181398,1.1569003999999998,-54.5211944444444,0.03542855267438075,41101.552734375,4,49325.0341796875,692934301,79.2051666666667,25665.1845703125,23517.341796875,79.15274000000002,-54.522467000000006,353.271914283719,37128.636,DES_Y3,1.1569003999999998,207.24304,0.8503200209682173,226.80347509844432,0.8503200209682173,226.80347509844418,79.15834368130022,-54.51454650034462,XMMXCSJ051636.6-543120.8,0.299,5.93553,5.821536052,6.05047411,7.855279456,7.768655616850883,7.951324018841962,2.11973067,2.107260101009745,2.1322217769550678,67.8504,63.3489
6,1,XMMXCS J041114.1-481910.9,XMMXCSJ041114.1-481910.9,4,4,62.795693300108006,-48.3276790985298,0.42028984,178.04501000000005,5.029322,1.1222923,-48.32925,0.013049658812522373,6495.39013671875,6,6491.949951171875,723160101,62.7762083333333,4961.818237304688,6010.1470947265625,62.809001,-48.319708,237.31286739525243,1721.9160000000006,DES_Y3,1.1222923,178.04501000000005,1.1855249756220514,393.72042140370337,1.1855249756220514,393.72042140370553,62.81891731360145,-48.31535319378601,XMMXCSJ041114.1-481910.9,0.42,5.96545,5.284333304,6.6481103,12.99346155,12.125737486639526,13.80903192481108,3.496990563,3.399755102954438,3.5966034363971,35.1315,32.6605


* We're interested in observations that are Detected == True. 

* We also want to remove observations that do not have `r2500_temperatures`

In [5]:
catalog_matcha[catalog_matcha['Detected'] == True]
catalog_matcha_detected = catalog_matcha[~np.isnan(catalog_matcha["r2500_temperature"])]

print(f"The number of columns for `catalog_matcha_detected` are {len(catalog_matcha_detected.colnames)}")
print(f"The number of rows are {len(catalog_matcha_detected)}.")
catalog_matcha_detected[0:3]

The number of columns for `catalog_matcha_detected` are 110
The number of rows are 205.


Name,MEM_MATCH_ID,Analyzed,Detected,redMaPPer_ra,redMaPPer_dec,Redshift,Obsids,lambda,lambda_err_low,lambda_err_high,band_lumin_upper_lim_undetected,band_lumin_upper_lim_detected,lumin_no_tx,lumin_no_tx_err_low,lumin_no_tx_err_high,x_ray_peak_ra,x_ray_peak_dec,500_kiloparsecs_ra,500_kiloparsecs_dec,500_kiloparsecs_temperature,500_kiloparsecs_temperature_err_low,500_kiloparsecs_temperature_err_high,500_kiloparsecs_band_lumin,500_kiloparsecs_band_lumin_err_low,500_kiloparsecs_band_lumin_err_high,500_kiloparsecs_bolo_lumin,500_kiloparsecs_bolo_lumin_err_low,500_kiloparsecs_bolo_lumin_err_high,500_kiloparsecs_radius,500_kiloparsecs_radius_err_low,500_kiloparsecs_radius_err_high,500_kiloparsecs_SNR,500_kiloparsecs_SNR_err_low,500_kiloparsecs_SNR_err_high,r2500_ra,r2500_dec,r2500_temperature,r2500_temperature_err_low,r2500_temperature_err_high,r2500_band_lumin,r2500_band_lumin_err_low,r2500_band_lumin_err_high,r2500_bolo_lumin,r2500_bolo_lumin_err_low,r2500_bolo_lumin_err_high,r2500_radius,r2500_radius_err_low,r2500_radius_err_high,r2500_SNR,r2500_SNR_err_low,r2500_SNR_err_high,r500_ra,r500_dec,r500_temperature,r500_temperature_err_low,r500_temperature_err_high,r500_band_lumin,r500_band_lumin_err_low,r500_band_lumin_err_high,r500_bolo_lumin,r500_bolo_lumin_err_low,r500_bolo_lumin_err_high,r500_radius,r500_radius_err_low,r500_radius_err_high,r500_SNR,r500_SNR_err_low,r500_SNR_err_high,r500_core_cropped_ra,r500_core_cropped_dec,r500_core_cropped_temperature,r500_core_cropped_temperature_err_low,r500_core_cropped_temperature_err_high,r500_core_cropped_band_lumin,r500_core_cropped_band_lumin_err_low,r500_core_cropped_band_lumin_err_high,r500_core_cropped_bolo_lumin,r500_core_cropped_bolo_lumin_err_low,r500_core_cropped_bolo_lumin_err_high,r500_core_cropped_radius,r500_core_cropped_radius_err_low,r500_core_cropped_radius_err_high,r500_core_cropped_SNR,r500_core_cropped_SNR_err_low,r500_core_cropped_SNR_err_high,peak_fix,masked,bad_mode,quick look bad,obsid_bad,merger,Serendipitous,edge_centroid,edge_peak,edge_r2500,edge_r500,edge_bkgd,overlap_r2500,overlap_r500,overlap_bkgd,notes,Group 1 Check,redmapper wrong,right RA,right DEC,right in top 5,right not member,ambiguous,Chandra Notes
bytes15,int64,bool,bool,float64,float64,float64,bytes96,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes4,int64,int64,bytes4,bytes26,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,bytes106,bytes14,int64,float64,float64,int64,int64,int64,bytes145
catalogue_1,2,True,True,83.231901970306,-37.0266894362176,0.2833377420902252,15112,199.432,5.87942,5.87942,nan,nan,4.5843e+44,4.815e+42,4.8632e+42,83.2325,-37.02649,83.23211,-37.02644,11.344,0.68483,0.68878,4.3399e+44,6.51e+42,6.5277e+42,1.7469e+45,2.866e+43,2.9201e+43,500.0,0.0,0.0,94.277,1.087,1.087,83.23188,-37.02643,11.166,0.63832,0.6424,4.944e+44,6.9038e+42,6.8134e+42,1.9782e+45,3.086e+43,3.0796e+43,672.57,23.048,22.571,101.61,1.093,1.093,83.23226,-37.02672,10.934,0.85751,0.85747,5.3675e+44,8.55e+42,8.5518e+42,2.1305e+45,4.5892e+43,4.6121e+43,1511.0,49.857,48.953,79.355,1.039,1.039,83.23226,-37.02672,12.746,1.8843,2.0198,2.7238e+44,6.0129e+42,8.2416e+42,1.1614e+45,6.8477e+43,2.9554e+43,1511.0,49.857,48.953,46.384,1.0189,1.0189,N/A,0,0,N/A,N/A,0,0,0,0,0,0,0,0,0,0,N/A,yes,0,9999.0,9999.0,0,0,0,N/A
catalogue_3,4,True,True,79.1556849306587,-54.5004588211563,0.29875507950782776,"9331, 15099",207.243,7.1814,7.1814,nan,nan,2.1914e+44,3.6

Here we join the two tables based on the common values in the `MEM_MATCH_ID` column from table `catalog_matcha_detected`.

In [6]:
catalog_join = join(catalog_matcha_detected, catalog_xmm, keys='MEM_MATCH_ID')

print(f"The number of columns for table `catalog_join` {len(catalog_join.colnames)}.")
catalog_join

The number of columns for table `catalog_join` 156.


Name,MEM_MATCH_ID,Analyzed,Detected,redMaPPer_ra,redMaPPer_dec,Redshift,Obsids,lambda,lambda_err_low,lambda_err_high,band_lumin_upper_lim_undetected,band_lumin_upper_lim_detected,lumin_no_tx,lumin_no_tx_err_low,lumin_no_tx_err_high,x_ray_peak_ra,x_ray_peak_dec,500_kiloparsecs_ra,500_kiloparsecs_dec,500_kiloparsecs_temperature,500_kiloparsecs_temperature_err_low,500_kiloparsecs_temperature_err_high,500_kiloparsecs_band_lumin,500_kiloparsecs_band_lumin_err_low,500_kiloparsecs_band_lumin_err_high,500_kiloparsecs_bolo_lumin,500_kiloparsecs_bolo_lumin_err_low,500_kiloparsecs_bolo_lumin_err_high,500_kiloparsecs_radius,500_kiloparsecs_radius_err_low,500_kiloparsecs_radius_err_high,500_kiloparsecs_SNR,500_kiloparsecs_SNR_err_low,500_kiloparsecs_SNR_err_high,r2500_ra,r2500_dec,r2500_temperature,r2500_temperature_err_low,r2500_temperature_err_high,r2500_band_lumin,r2500_band_lumin_err_low,r2500_band_lumin_err_high,r2500_bolo_lumin,r2500_bolo_lumin_err_low,r2500_bolo_lumin_err_high,r2500_radius,r2500_radius_err_low,r2500_radius_err_high,r2500_SNR,r2500_SNR_err_low,r2500_SNR_err_high,r500_ra,r500_dec,r500_temperature,r500_temperature_err_low,r500_temperature_err_high,r500_band_lumin,r500_band_lumin_err_low,r500_band_lumin_err_high,r500_bolo_lumin,r500_bolo_lumin_err_low,r500_bolo_lumin_err_high,r500_radius,r500_radius_err_low,r500_radius_err_high,r500_SNR,r500_SNR_err_low,r500_SNR_err_high,r500_core_cropped_ra,r500_core_cropped_dec,r500_core_cropped_temperature,r500_core_cropped_temperature_err_low,r500_core_cropped_temperature_err_high,r500_core_cropped_band_lumin,r500_core_cropped_band_lumin_err_low,r500_core_cropped_band_lumin_err_high,r500_core_cropped_bolo_lumin,r500_core_cropped_bolo_lumin_err_low,r500_core_cropped_bolo_lumin_err_high,r500_core_cropped_radius,r500_core_cropped_radius_err_low,r500_core_cropped_radius_err_high,r500_core_cropped_SNR,r500_core_cropped_SNR_err_low,r500_core_cropped_SNR_err_high,peak_fix,masked,bad_mode,quick look bad,obsid_bad,merger,Serendipitous,edge_centroid,edge_peak,edge_r2500,edge_r500,edge_bkgd,overlap_r2500,overlap_r500,overlap_bkgd,notes,Group 1 Check,redmapper wrong,right RA,right DEC,right in top 5,right not member,ambiguous,Chandra Notes,Target,XCS_NAMES,XCS_NAMES_2,col1,Unnamed: 0.1,RA,DEC,Z_1,LAMBDA_CHISQ,LAMBDA_CHISQ_E,R_LAMBDA,DEC_xmm,DISTANCE,EXP,ID,MEDIAN_EXP,OBSID,RA_xmm,WORST_EXP,WORST_MEDIAN_EXP,RA_ext,DEC_ext,P_DIS,COUNTS,SURVEY,R_LAM,LAM,Pk2RM,Pk2RM_kpc,Pk2RM2,Pk2RM2_kpc,RA_pk,DEC_pk,#XCS_NAME,z_2,Tx,Tx_m,Tx_p,Lx,Lx_m,Lx_p,Lx52,Lx52_m,Lx52_p,pnsn,pnsnr_alt
bytes15,int64,bool,bool,float64,float64,float64,bytes96,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes4,int64,int64,bytes4,bytes26,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,bytes106,bytes14,int64,float64,float64,int64,int64,int64,bytes145,int32,str25,str24,int32,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,float64,int32,float64,float64,float64,float64,float64,float64,float64,str6,float64,float64,float64,float64,float64,float64,float64,float64,str24,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
catalogue_1,2,True,True,83.231901970306,-37.0266894362176,0.2833377420902252,15112,199.432,5.87942,5.87942,nan,nan,4.5843e+44,4.815e+42,4.8632e+42,83.2325,-37.02649,83.23211,-37.02644,11.344,0.68483,0.68878,4.3399e+44,6.51e+42,6.5277e+42,1.7469e+45,2.866e+43,2.920

In [7]:
"""
Before starting make sure indeces of df that
    you're using are from 0-->n. Otherwise,
    the rows will not be placed correctly 
    in the new df. 
"""

#Create empty dataframe.
catalog_xmm_matcha = Table()

# Include `Catalog Name` and `Mem Match Id`
catalog_xmm_matcha["Name"] = catalog_join["Name"]
catalog_xmm_matcha["MEM_MATCH_ID"] = catalog_join["MEM_MATCH_ID"]
catalog_xmm_matcha["Redshift"] = catalog_join["Redshift"]

#Below is the r2500 temperatures from matcha catalog
catalog_xmm_matcha["Tx_r2500_matcha"] = catalog_join["r2500_temperature"]
catalog_xmm_matcha["Tx_r2500_err_high_matcha"] = catalog_join["r2500_temperature_err_high"]
catalog_xmm_matcha["Tx_r2500_err_low_matcha"] = catalog_join["r2500_temperature_err_low"]

#Below is the r500 temperatures from mat catalog
catalog_xmm_matcha["Tx_r500_matcha"] = catalog_join["r500_temperature"]
catalog_xmm_matcha["Tx_r500_err_high_matcha"] = catalog_join["r500_temperature_err_high"]
catalog_xmm_matcha["Tx_r500_err_low_matcha"] = catalog_join["r500_temperature_err_low"]

#Below is the Tx from XMM catalog
catalog_xmm_matcha["Tx_xmm"] = catalog_join["Tx"]
catalog_xmm_matcha["Tx_err_high_xmm"] = catalog_join["Tx_p"] - catalog_join["Tx"]
catalog_xmm_matcha["Tx_err_low_xmm"] = catalog_join["Tx"] - catalog_join["Tx_m"]

#Below is the SNR columns from both catalogs
catalog_xmm_matcha["r2500_SNR_matcha"] = catalog_join["r2500_SNR"]
catalog_xmm_matcha["r500_SNR_matcha"] = catalog_join["r500_SNR"]
catalog_xmm_matcha["r2500_PNSNR_alt_xmm"] = catalog_join["pnsnr_alt"]

#Show table with dimensions.
print(f"The dimensions of df_xmm_matcha are {len(catalog_xmm_matcha.colnames)}.")
catalog_xmm_matcha

The dimensions of df_xmm_matcha are 15.


Name,MEM_MATCH_ID,Redshift,Tx_r2500_matcha,Tx_r2500_err_high_matcha,Tx_r2500_err_low_matcha,Tx_r500_matcha,Tx_r500_err_high_matcha,Tx_r500_err_low_matcha,Tx_xmm,Tx_err_high_xmm,Tx_err_low_xmm,r2500_SNR_matcha,r500_SNR_matcha,r2500_PNSNR_alt_xmm
bytes15,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
catalogue_1,2,0.2833377420902252,11.166,0.6424,0.63832,10.934,0.85747,0.85751,7.828,0.2203031969999989,0.21822615200000062,101.61,79.355,50.3601
catalogue_3,4,0.29875507950782776,10.675,0.82221,0.81572,14.845,2.3899,1.1838,5.93553,0.11494410999999971,0.11399394800000007,80.537,73.403,63.3489
catalogue_5,6,0.41268131136894226,7.1066,0.31834,0.31181,nan,nan,nan,5.96545,0.6826603000000002,0.6811166960000001,84.785,nan,32.6605
catalogue_7,8,0.2983894646167755,9.8859,0.25544,0.25569,9.6186,0.35822,0.21636,8.29323,0.21034413799999996,0.20948681800000024,251.93,188.34,57.1254
catalogue_14,16,0.37259918451309204,9.117,1.8887,1.2807,8.1848,1.9394,1.3461,8.18501,0.1535090540000006,0.15274920300000083,32.063,25.5,81.5344
catalogue_20,24,0.2382102757692337,12.27,1.3641,0.95964,15.041,2.378,2.2175,9.08992,0.33020290500000016,0.32942418399999873,75.774,58.134,57.0353
catalogue_21,25,0.2831774652004242,9.2817,0.56765,0.45277,9.0506,0.61982,0.59182,7.45213,0.23026912699999968,0.2280244050000002,95.613,73.676,48.0936
catalogue_24,29,0.5994992852210999,13.626,0.51466,0.51398,nan,nan,nan,12.4299,0.07564686000000087,0.07560574999999936,162.04,nan,213.98
catalogue_36,43,0.21676212549209595,4.5171,0.17414,0.17101,4.4657,0.203,0.20082,4.06911,0.07421478399999959,0.07358307100000028,82.441,76.631,58.6387


Great! Now we have our new catalog. Let's save it!

In [ ]:
catalog_xmm_matcha.write("filepath/filename.fits", format="fits")

### Add in column to matcha catalog

* Has mem_match_id in common `Tx_in_xmm`

In [8]:
bucket = np.isin(catalog_matcha["MEM_MATCH_ID"], catalog_xmm["MEM_MATCH_ID"])

print(f"The number of observation in common between catalogs are {sum(bucket)}.")

The number of observation in common between catalogs are 63.


In [ ]:
catalog_matcha["Tx_in_xmm"] = bucket.astype(np.int64)

catalog_matcha[0:5]

In [ ]:
catalog_matcha.write("filepath/filename.fits", format='fits')

Done!